# Training

In [1]:
# Automatic reloading
%load_ext autoreload
%autoreload 2

In [2]:
from src.data_preparation import *
from src.models import *
from src.train_eval import *

from torch.utils.data import DataLoader

In [3]:
train_dataset, val_dataset, test_dataset, norm_stats = prepare_datasets(
    sequence_size=1000,
    stride=25,
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42,
    load_if_exists=True
)

✅ HARDataset object, normalization statistics, and split indices loaded from saved_components
✅ Created train, validation, and test datasets (normalized to train set)


In [4]:
train_dataloader = DataLoader(train_dataset, batch_size = 64, shuffle= True)
validation_dataloader = DataLoader(val_dataset, batch_size = 64, shuffle= True)
test_dataloader = DataLoader(test_dataset, batch_size = 64, shuffle= True)

In [5]:
model = HarGRU()
# model.load_state_dict(torch.load("models/HarGRU_2025-03-11_19-11-04/HarGRU_epoch100.pth"))
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

HarGRU model loaded on cuda.


In [6]:
training_loss_history, validation_loss_history, accuracy_history, f1_history, precision_history, recall_history = train_HAR70_model(
    model, 
    optimizer, 
    train_dataloader, 
    validation_dataloader, 
    num_epochs = 5
)

Epoch [1/5] | Time: 36.99s
(Training) Loss: 0.0061
(Validation) Loss: 0.0035, Accuracy: 0.5529, F1: 0.5406, Precision: 0.5302, Recall: 0.5529
Epoch [2/5] | Time: 32.56s
(Training) Loss: 0.0033
(Validation) Loss: 0.0028, Accuracy: 0.5570, F1: 0.5466, Precision: 0.5374, Recall: 0.5570
Epoch [3/5] | Time: 33.83s
(Training) Loss: 0.0027
(Validation) Loss: 0.0024, Accuracy: 0.5600, F1: 0.5542, Precision: 0.5825, Recall: 0.5600
Epoch [4/5] | Time: 34.84s
(Training) Loss: 0.0023
(Validation) Loss: 0.0022, Accuracy: 0.5768, F1: 0.5838, Precision: 0.6275, Recall: 0.5768
Epoch [5/5] | Time: 32.52s
(Training) Loss: 0.0022
(Validation) Loss: 0.0022, Accuracy: 0.5946, F1: 0.6087, Precision: 0.6833, Recall: 0.5946
✅ Model saved: models\HarGRU_2025-03-11_22-21-42\HarGRU_epoch5.pth


In [7]:
save_training_plots_and_metric_history(training_loss_history, validation_loss_history, accuracy_history, f1_history, precision_history, recall_history)

✅ Plots saved to: results\2025-03-11_22-24-33
✅ Metric histories saved to: results\2025-03-11_22-24-33\metric_histories.pth


In [8]:
# Test metrics
loss, accuracy, f1, precision, recall, conf_matrix = evaluate_HAR70_model(model, test_dataloader)
print(f"(Test) Loss: {loss:.4f}, Accuracy: {accuracy:.4f}, F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

(Test) Loss: 0.0023, Accuracy: 0.5889, F1: 0.5979, Precision: 0.6204, Recall: 0.5889
